In [ ]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict
import sys

In [ ]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

# model = SwinTransformer3D(**cfg)

In [ ]:
def get_model(num_classes,cfg, shape_of_input=(10,3,32,224,224)):
    inputs = tf.keras.Input(shape_of_input[1:])
    backbone = SwinTransformer3D(**cfg, shape_of_input=shape_of_input)
    x = backbone(inputs, training= False)
    print("backbone", x.shape)
    outputs = I3DHead_tf(num_classes, 768, training=True)(x)
    return tf.keras.Model(inputs, outputs)

In [56]:
shape_of_input = (5, 3, 8, 112,112)
model = get_model(5, cfg, shape_of_input=shape_of_input)

X = tf.random.normal(shape_of_input)
y = tf.random.uniform((shape_of_input[0],1), 0, 5, tf.dtypes.int32)
model.summary()

shape_of_input:  (5, 3, 8, 112, 112)
backbone (None, 768, 4, 4, 4)
5 768 {'type': 'CrossEntropyLoss'} avg 0.5 0.01 {'training': True}
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 3, 8, 112, 112)]  0         
                                                                 
 swin_transformer3d_6 (SwinT  (None, 768, 4, 4, 4)     29694438  
 ransformer3D)                                                   
                                                                 
Total params: 29,694,438
Trainable params: 27,850,470
Non-trainable params: 1,843,968
_________________________________________________________________


In [57]:
# inputs = tf.keras.Input(shape_of_input[1:])
# model(inputs, training=True)

In [58]:
y1 = model.predict(X)
y1

2022-09-06 14:23:54.827449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 7s 7s/step


array([[[[[-2.96648368e-02, -7.16615915e-02, -1.67777270e-01,
            4.86763030e-01],
          [-1.01131648e-01,  1.05203390e+00,  1.05343199e+00,
            1.06250000e+00],
          [ 9.58439261e-02,  8.84916842e-01,  6.87209666e-02,
            1.34724528e-01],
          [ 3.63915637e-02,  4.66655076e-01,  3.37274015e-01,
           -8.59263539e-02]],

         [[ 4.10968661e-01,  3.37311149e-01,  7.37256110e-01,
            2.45267048e-01],
          [ 3.02936643e-01, -1.01288959e-01, -1.80549920e-01,
            6.70054376e-01],
          [-3.09125692e-01, -1.14763707e-01,  4.77106214e-01,
            1.93650082e-01],
          [-1.02631710e-01,  3.13750386e-01,  1.26069570e+00,
            5.22891402e-01]],

         [[ 5.64833462e-01,  2.17401981e-01,  7.17733502e-01,
            1.76471263e-01],
          [ 2.63320535e-01, -3.84076416e-01,  3.35332811e-01,
            5.77438653e-01],
          [ 3.48050416e-01,  3.26116979e-01, -6.15864843e-02,
            5.81660390e-

In [53]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(), metrics=["accuracy"])

model.fit(X, y, epochs=1)

2022-09-06 14:22:58.832567: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 18s 18s/step - loss: 2.8406 - accuracy: 0.1991


In [55]:
y2 = model(X, training=False)
y2

<tf.Tensor: shape=(5, 768, 4, 4, 4), dtype=float32, numpy=
array([[[[[nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan]],

         [[nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan]],

         [[nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan]],

         [[nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan]]],


        [[[nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan]],

         [[nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan]],

         [[nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan],
          [nan, nan, nan, nan]],

         [[nan, nan, nan, nan

In [ ]:
# model.layers[1].layers[5].weights